## Selector Group Chat
`SelectorGroupChat` implements a team where participants take turns broadcasting messages to all other participants, with the next speaker selected by a generative model (e.g., an LLM) based on the shared context. This enables dynamic and context-aware multi-agent collaboration.

SelectorGroupChat provides several key features:

- Model-based speaker selection

- Configurable participant roles and descriptions

- Optional prevention of consecutive turns by the same speaker

- Customizable selection prompting

- Customizable selection function to override the default model-based selection

### How does it work?
SelectorGroupChat is a group chat similar to RoundRobinGroupChat, but with a model-based next speaker selection mechanism. When the team receives a task through run() or run_stream(), the following steps are executed:

- The team analyzes the current conversation context, including the conversation history and participants’ name and description attributes, to determine the next speaker using a model. You can override the model by providing a custom selection function.

- The team prompts the selected speaker agent to provide a response, which is then broadcasted to all other participants.

- The termination condition is checked to determine if the conversation should end, if not, the process repeats from step 1.

- When the conversation ends, the team returns the TaskResult containing the conversation history from this task.

Once the team finishes the task, the conversation context is kept within the team and all participants, so the next task can continue from the previous conversation context. You can reset the conversation context by calling `reset()`.

In this section, we will demonstrate how to use `SelectorGroupChat` with a simple example for a web search and data analysis task.

## Web Search and Analysis Example

### Agents
<Image src='docs/selector-group-chat.svg'>

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

print(os.getenv('CHAT_MODEL'))

gpt-4o-mini


In [2]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["AIPROJECT_CONNECTION_STRING"]
)

base_url = project_client.inference.get_azure_openai_client(
    api_version="2024-06-01").base_url

api_endpoint = f'https://{base_url.host}/'

api_key = project_client.inference.get_azure_openai_client(
    api_version="2024-06-01").api_key

deployment_name = os.environ["CHAT_MODEL"]

aoai_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=api_endpoint,
    model="gpt-4o-mini",
    azure_deployment=deployment_name,
    api_key=api_key,
    api_version="2024-06-01"
)

In [3]:
# Note: This example uses mock tools instead of real APIs for demonstration purposes
def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."


def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100

In [4]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat


planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=aoai_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        Web search agent: Searches for information
        Data analyst: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="A web search agent.",
    tools=[search_web_tool],
    model_client=aoai_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_tool - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="A data analyst agent. Useful for performing calculations.",
    model_client=aoai_client,
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
    """,
)

In [5]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=aoai_client,
    termination_condition=termination,
)

In [6]:
from autogen_agentchat.ui import Console


task = "Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- user ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- WebSearchAgent ----------
[FunctionCall(id='call_p6Y5WQ7WabUTA2hMK23YFwU1', arguments='{"query": "Miami Heat player highest points 2006-2007 season"}', name='search_web_tool'), FunctionCall(id='call_7Ov5KF1V5wIj3FLEXkGjygpi', arguments='{"query": "Miami Heat player total rebounds 2007-2008 season stats"}', name='search_web_tool'), FunctionCall(id='call_FdHcEYnpWFG4DHJO1Aewl7B6', arguments='{"query": "Miami Heat player total rebounds 2008-2009 season stats"}', name='search_web_tool')]
---------- WebSearchAgent ----------
[FunctionExecutionResult(content='Here are the total points scored by Miami Heat players in the 2006-2007 season:\n        Udonis Haslem: 844 points\n        Dwayne Wade: 1397 points\n        James Posey: 550 points\n        ...\n        ', name='search_w

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), ToolCallRequestEvent(source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=142, completion_tokens=97), metadata={}, content=[FunctionCall(id='call_p6Y5WQ7WabUTA2hMK23YFwU1', arguments='{"query": "Miami Heat player highest points 2006-2007 season"}', name='search_web_tool'), FunctionCall(id='call_7Ov5KF1V5wIj3FLEXkGjygpi', arguments='{"query": "Miami Heat player total rebounds 2007-2008 season stats"}', name='search_web_tool'), FunctionCall(id='call_FdHcEYnpWFG4DHJO1Aewl7B6', arguments='{"query": "Miami Heat player total rebounds 2008-2009 season stats"}', name='search_web_tool')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='WebSearchAgent', models_usage=None, metadata={}

In [7]:
await team.reset()  # Reset the team for the next run.

## Custom Selector Function
Often times we want better control over the selection process. To this end, we can set the `selector_func` argument with a custom selector function to override the default model-based selection. For instance, we want the Planning Agent to speak immediately after any specialized agent to check the progress.

In [8]:
from typing import Sequence
from autogen_agentchat.messages import ChatMessage


def selector_func(messages: Sequence[ChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None


team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=aoai_client,
    termination_condition=termination,
    selector_func=selector_func,
)

await Console(team.run_stream(task=task))

---------- user ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- PlanningAgent ----------
To address your request, I will break it down into two main subtasks:

1. Web search agent: Find out who the Miami Heat player was with the highest points in the 2006-2007 season.
2. Data analyst: Calculate the percentage change in the total rebounds for that player between the 2007-2008 and 2008-2009 seasons.

Let’s start the planning:

1. Web search agent: Search for the Miami Heat player who scored the highest points in the 2006-2007 season.
2. Data analyst: Find the total rebounds for the above player in the 2007-2008 and 2008-2009 seasons and calculate the percentage change. 

Now, I will delegate these tasks. 

1. Web search agent : Find out who the Miami Heat player had the highest points in the 2006-2007 season.
2. Data analyst : Find the

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=159, completion_tokens=225), metadata={}, content='To address your request, I will break it down into two main subtasks:\n\n1. Web search agent: Find out who the Miami Heat player was with the highest points in the 2006-2007 season.\n2. Data analyst: Calculate the percentage change in the total rebounds for that player between the 2007-2008 and 2008-2009 seasons.\n\nLet’s start the planning:\n\n1. Web search agent: Search for the Miami Heat player who scored the highest points in the 2006-2007 season.\n2. Data analyst: Find the total rebounds for the above player in the 2007-2008 and 2008-2009 seasons and calculate the perc